# 💻 LMFast: Code Generation & Agents

**Train specialized coding assistants!**

## What You'll Learn
- Fine-tune on code datasets (Python, JS, SQL)
- Understand "infilling" vs "instruction"
- Build a `CodeAgent` that writes and executes code
- Evaluate using HumanEval (intro)

**Time to complete:** ~15 minutes

## 1️⃣ Setup

In [ ]:
!pip install -q lmfast[all]

import lmfast
lmfast.setup_colab_env()

import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Prepare Code Data

Code data usually is comprised of raw source files or instruction-response pairs.

In [ ]:
from datasets import Dataset

# Example Python instruction dataset
data = [
    {
        "instruction": "Write a function to calculate fibonacci numbers.",
        "output": "def fib(n):\n    if n <= 1: return n\n    return fib(n-1) + fib(n-2)"
    },
    {
        "instruction": "Create a pandas dataframe from a dict.",
        "output": "import pandas as pd\ndf = pd.DataFrame({'a': [1,2], 'b': [3,4]})"
    }
]

dataset = Dataset.from_list(data)

# Format
def format_code(x):
    return {"text": f"# {x['instruction']}\n{x['output']}"}

formatted_ds = dataset.map(format_code)
print(formatted_ds[0]['text'])

## 3️⃣ Train a Code Model

We start with a base model that already knows some code well (e.g. Qwen2.5-0.5B-Coder or SmolLM).

In [ ]:
from lmfast import train

trainer = train(
    model="HuggingFaceTB/SmolLM-360M-Instruct",
    dataset=formatted_ds,
    output_dir="./code_assistant",
    max_steps=30,
    learning_rate=2e-4
)

print("✅ Code training complete (simulated)")

## 4️⃣ Using `CodeAgent`

The `CodeAgent` is a specialized agent that can generate AND execute Python code safely (in a sandbox if configured).

In [ ]:
from lmfast.agents.specialized import CodeAgent
from lmfast.inference import SLMServer

model = SLMServer("./code_assistant")

# Agent that can run python
agent = CodeAgent(
    model_generate_fn=model.generate,
    unsafe_mode=True # Example only! Sandbox usually recommended.
)

task = "Calculate the 10th fibonacci number and print it."
result = agent.run(task)

print(f"Result: {result}")

## 🎉 Summary

You've learned how to:
- ✅ Format code instruction data
- ✅ Train a specialized code model
- ✅ Use `CodeAgent` to execute generated code

### Next Steps
- Try fine-tuning on SQL data for a Text-to-SQL agent!